
# Multi-Agent Tourism Planner — CrewAI + Ollama (phi4) (PT/EN)

Este notebook cria um **pipeline de agentes CrewAI** para planejamento de viagens (Pesquisa da cidade → Logística → Itinerário → Dicas/Idioma) usando **Ollama local** com o modelo **`phi4`** via LangChain (`ChatOllama`).  
Você pode alternar **Português/Inglês** nos prompts.

> Requisitos: Python 3.10+, Ollama instalado e rodando localmente, modelo `phi4` baixado (`ollama pull phi4`).

---

**How it works (EN):** This notebook sets up a **CrewAI multi-agent pipeline** for travel planning (City Research → Logistics → Itinerary → Tips/Language) using **local Ollama** with **`phi4`** via LangChain (`ChatOllama`).  
You can switch **Portuguese/English** prompts.



## 1) Setup

- Start Ollama locally and pull the model:
```bash
ollama pull phi4
ollama serve  # if not running already
```
- (Optional) Install Python deps in this environment:
```bash
pip install crewai==0.51.1 langchain==0.2.16 langchain-community==0.2.11 pydantic==2.8.2 python-dotenv==1.0.1
```


In [ ]:

import os
from typing import List, Dict, Any

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

from crewai import Agent, Task, Crew, Process

# Optional: environment variables via .env (ignored if not present)
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# ---- LLM configuration ----
MODEL_NAME = os.getenv("LLM_MODEL_NAME", "phi4")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
LLM_TEMPERATURE = float(os.getenv("LLM_TEMPERATURE", "0.1"))

# Global singleton for the LLM instance (similar to your original Streamlit pattern)
_llm_instance = None

def load_llm() -> ChatOllama:
    global _llm_instance
    if _llm_instance is None:
        _llm_instance = ChatOllama(
            model=MODEL_NAME.lower().replace(" ", ""),
            base_url=OLLAMA_BASE_URL,
            temperature=LLM_TEMPERATURE,
        )
    return _llm_instance

def model_response(system_prompt: str, user_query: str, chat_history: List[List[str]] | None = None) -> str:
    """Single-turn style utility compatible with your previous function."""
    llm = load_llm()
    messages = [("system", system_prompt)]
    if chat_history:
        messages += chat_history
    messages.append(("user", user_query))
    prompt_template = ChatPromptTemplate.from_messages(messages)
    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"input": user_query})



## 2) Bilingual Prompts (PT/EN)

Selecione o idioma dos agentes definindo `LANG = "pt"` ou `LANG = "en"`.


In [ ]:

LANG = "pt"  # "pt" or "en"

PROMPTS = {
    "pt": {
        "system_base": (
            "Você é um assistente de planejamento de viagens composto por múltiplos agentes especializados. "
            "Trabalhe com precisão, cite suposições explícitas quando faltarem dados, e produza respostas concisas e acionáveis."
        ),
        "researcher": (
            "Atue como Pesquisador(a) da Cidade. Reúna fatos essenciais e verificáveis sobre a cidade/país/atrações: "
            "contexto, clima geral da época, regiões, segurança, transporte público, bairros recomendados, custos médios, "
            "e principais pontos de interesse alinhados aos interesses do viajante."
        ),
        "logistics": (
            "Atue como Planejador(a) de Logística. Converta as preferências e datas em um plano logístico: "
            "opções de chegada/saída (voos/terrestre), deslocamentos intraurbanos, estimativa de custos por dia/faixa, "
            "ordenação dos bairros/regiões por conveniência, e observações de tempo de deslocamento."
        ),
        "itinerary": (
            "Atue como Designer de Itinerário. Construa um roteiro diário claro, com blocos de manhã/tarde/noite, "
            "juntando atrações, tempo de deslocamento aproximado, janelas de alimentação, e alternativas em caso de clima ruim. "
            "Mantenha duração realista, respeitando perfil e orçamento."
        ),
        "tips": (
            "Atue como Especialista em Dicas e Idioma. Liste dicas práticas (gorjetas, plugues, cartões/saque, segurança, "
            "feriados, recomendações culinárias) e um mini-guia de frases úteis no idioma local com pronúncia aproximada."
        ),
        "final_instructions": (
            "Consolide em formato: 1) Sumário do contexto; 2) Plano logístico; 3) Itinerário dia-a-dia; 4) Dicas e idioma. "
            "Use listas curtas, destaque horários, estime custos em faixas e indique suposições."
        ),
    },
    "en": {
        "system_base": (
            "You are a travel-planning assistant composed of multiple specialized agents. "
            "Work accurately, state assumptions explicitly when data is missing, and produce concise, actionable output."
        ),
        "researcher": (
            "Act as the City Researcher. Gather essential verifiable facts about the city/country/attractions: "
            "context, typical weather for the dates, districts, safety, public transit, recommended areas, average costs, "
            "and key points of interest aligned with the traveler’s interests."
        ),
        "logistics": (
            "Act as the Logistics Planner. Convert preferences and dates into a logistics plan: "
            "arrival/departure options (air/ground), intra-city transport, cost estimates per day/range, "
            "ordering districts/areas by convenience, and notes on travel times."
        ),
        "itinerary": (
            "Act as the Itinerary Designer. Build a clear day-by-day plan with morning/afternoon/evening blocks, "
            "grouping attractions, approximate travel times, meal windows, and rain-day alternatives. "
            "Keep it realistic and aligned to profile and budget."
        ),
        "tips": (
            "Act as the Tips & Language Specialist. Provide practical tips (tipping, plugs, cards/cash, safety, holidays, "
            "food recommendations) and a mini phrasebook in the local language with approximate pronunciation."
        ),
        "final_instructions": (
            "Consolidate as: 1) Context summary; 2) Logistics plan; 3) Day-by-day itinerary; 4) Tips & language. "
            "Use short lists, highlight times, estimate costs in ranges, and state assumptions."
        ),
    },
}



## 3) Agent & Task Factory (CrewAI)


In [ ]:

def build_agents(language: str = LANG) -> Dict[str, Agent]:
    llm = load_llm()
    p = PROMPTS[language]
    system = p["system_base"]

    city_researcher = Agent(
        role="City Researcher" if language=="en" else "Pesquisador(a) da Cidade",
        goal="Collect accurate city context aligned with traveler preferences.",
        backstory="Experienced travel analyst who synthesizes reliable, current information without external tools.",
        llm=llm,
        allow_delegation=False,
        verbose=False,
        memory=True,
        system_prompt=system + " " + p["researcher"],
    )

    logistics_planner = Agent(
        role="Logistics Planner" if language=="en" else "Planejador(a) de Logística",
        goal="Turn preferences/dates into a practical movement & cost plan.",
        backstory="Senior trip architect optimizing time, movement and budget.",
        llm=llm,
        allow_delegation=False,
        verbose=False,
        memory=True,
        system_prompt=system + " " + p["logistics"],
    )

    itinerary_designer = Agent(
        role="Itinerary Designer" if language=="en" else "Designer de Itinerário",
        goal="Create feasible, paced day-by-day itineraries.",
        backstory="Itinerary maker focused on realistic pacing and alternatives.",
        llm=llm,
        allow_delegation=False,
        verbose=False,
        memory=True,
        system_prompt=system + " " + p["itinerary"],
    )

    tips_language = Agent(
        role="Tips & Language Specialist" if language=="en" else "Especialista em Dicas e Idioma",
        goal="Provide practical tips and a compact phrasebook.",
        backstory="Cultural navigator providing pragmatic advice and language snippets.",
        llm=llm,
        allow_delegation=False,
        verbose=False,
        memory=True,
        system_prompt=system + " " + p["tips"],
    )

    return {
        "researcher": city_researcher,
        "logistics": logistics_planner,
        "itinerary": itinerary_designer,
        "tips": tips_language,
    }

def build_tasks(agents: Dict[str, Agent], inputs: Dict[str, Any], language: str = LANG) -> List[Task]:
    lang = language
    city = inputs.get("city", "")
    country = inputs.get("country", "")
    dates = inputs.get("dates", "")
    budget = inputs.get("budget", "moderate")
    interests = inputs.get("interests", [])
    travelers = inputs.get("travelers", "1 adult")
    profile = inputs.get("profile", "general")
    constraints = inputs.get("constraints", "none")

    if lang == "pt":
        interest_line = ", ".join(interests) if interests else "sem preferências específicas"
        base_ctx = (
            f"Cidade: {city}, País/Região: {country}, Datas: {dates}, Orçamento: {budget}, "
            f"Viajantes: {travelers}, Perfil: {profile}, Restrições: {constraints}. "
            f"Interesses: {interest_line}."
        )
        final_note = PROMPTS[lang]["final_instructions"]

        t1_desc = f"Use o contexto a seguir para pesquisa da cidade:\n{base_ctx}"
        t2_desc = f"Converta preferências e datas em um plano logístico pragmático.\n{base_ctx}"
        t3_desc = f"Crie um itinerário diário factível (manhã/tarde/noite).\n{base_ctx}"
        t4_desc = f"Liste dicas práticas e um mini-guia de frases.\n{base_ctx}"

        output_format = (
            "Formato de saída obrigatório (markdown):\n"
            "1) **Sumário do contexto**\n"
            "2) **Plano logístico**\n"
            "3) **Itinerário (dia a dia)**\n"
            "4) **Dicas e idioma**\n"
            "Inclua estimativas de custos por faixa e suposições explícitas."
        )
    else:
        interest_line = ", ".join(interests) if interests else "no specific preferences"
        base_ctx = (
            f"City: {city}, Country/Region: {country}, Dates: {dates}, Budget: {budget}, "
            f"Travelers: {travelers}, Profile: {profile}, Constraints: {constraints}. "
            f"Interests: {interest_line}."
        )
        final_note = PROMPTS[lang]["final_instructions"]

        t1_desc = f"Use the context below for city research:\n{base_ctx}"
        t2_desc = f"Convert preferences and dates into a pragmatic logistics plan.\n{base_ctx}"
        t3_desc = f"Create a feasible day-by-day itinerary (morning/afternoon/evening).\n{base_ctx}"
        t4_desc = f"Provide practical tips and a mini phrasebook.\n{base_ctx}"

        output_format = (
            "Required output format (markdown):\n"
            "1) **Context summary**\n"
            "2) **Logistics plan**\n"
            "3) **Day-by-day itinerary**\n"
            "4) **Tips & language**\n"
            "Include cost ranges and explicit assumptions."
        )

    tasks = [
        Task(description=t1_desc, agent=agents["researcher"], expected_output="City context summary in bullet points."),
        Task(description=t2_desc, agent=agents["logistics"], expected_output="Logistics plan with transport & cost ranges."),
        Task(description=t3_desc, agent=agents["itinerary"], expected_output="Paced day-by-day itinerary."),
        Task(description=t4_desc, agent=agents["tips"], expected_output="Practical tips + mini phrasebook."),
    ]

    # Final consolidation task handled by the Itinerary Designer for coherence
    tasks.append(
        Task(
            description=final_note + "\n" + output_format,
            agent=agents["itinerary"],
            expected_output="Consolidated markdown covering all sections.",
        )
    )
    return tasks

def run_crew(inputs: Dict[str, Any], language: str = LANG) -> str:
    agents = build_agents(language=language)
    tasks = build_tasks(agents, inputs, language=language)
    crew = Crew(agents=list(agents.values()), tasks=tasks, process=Process.sequential, verbose=False)
    result = crew.kickoff()
    return str(result)



## 4) Example Run

Ajuste os parâmetros e rode a célula para gerar o plano.


In [ ]:

example_inputs = {
    "city": "Buenos Aires",
    "country": "Argentina",
    "dates": "2025-08-22 to 2025-08-30",
    "budget": "moderate",
    "interests": ["architecture", "steakhouse", "tango", "museums"],
    "travelers": "2 adults",
    "profile": "likes walking, photography",
    "constraints": "avoid long bus rides",
}

LANG = "pt"  # or "en"
result_markdown = run_crew(example_inputs, language=LANG)
print(result_markdown)



## 5) Single-Call Chat Utility (Optional)

Função compatível com sua versão anterior (`model_response`) para perguntas diretas usando o mesmo LLM local.


In [ ]:

system_pt = PROMPTS["pt"]["system_base"]
reply = model_response(system_pt, "Liste 5 bairros interessantes em Buenos Aires para turistas com orçamento moderado.")
print(reply)



## 6) Notas & Troubleshooting

- Se o Ollama não estiver rodando, inicie com `ollama serve`.
- Baixe o modelo uma vez: `ollama pull phi4`.
- Ajuste `OLLAMA_BASE_URL` se sua instância não estiver no `localhost:11434`.
- `memory=True` nos agentes permite que cada agente retenha o contexto dentro da execução do Crew.
- Este notebook não usa ferramentas externas (APIs) para manter a execução **100% local**.



## 7) Referências

- CrewAI: https://docs.crewai.com/
- LangChain `ChatOllama`: https://python.langchain.com/docs/integrations/chat/ollama
- Ollama: https://ollama.com/
- Phi-4 model card (Ollama): https://ollama.com/library/phi4
